#### Initialize Libraries

In [ ]:
from os import listdir
from os.path import isfile, join
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import itertools

!pip install fasttext
import fasttext

data_path = ''
import nest_asyncio
from twitchio.ext import commands
from twitchio.abcs import Messageable

data_path = 'Data/'
results_path = 'Results/'
dataset_path = data_path+'Datasets/'
train_path = data_path+'Train/'
test_path = data_path+'Test/'


#### Auxiliar functions

In [ ]:
def repeated_chars(word):
    clean_word=""
    for _, dups in (itertools.groupby(word)):
        seq = list(dups)
        clean_word += seq[0]
        if len(seq)>1:
            clean_word += seq[1]
    return clean_word

def cleanComment(comment):
    stemming = PorterStemmer()
    stops = set(stopwords.words("spanish"))
    ## START CODE
    line= comment.lower() ## Transform in lowercase
    line=re.sub('[:\[\]&%$\"\'!./,;:?=¿^\-#_*+)<>(¡@]','',line)
    line= line.split() ## Tokenize the text to get a list of terms
    line= [repeated_chars(word) for word in line]
    #line= [word for word in line if word not in stops]  ##eliminate the stopwords (HINT: use List Comprehension)
    line= [stemming.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    cleanedComment = ""
    for i, word in enumerate(line):
      if i!=0:
        cleanedComment += " "
      cleanedComment += word
    return cleanedComment

no pero tien culazo


## FASTTEXT 

### Load Dataset

#### Read Categorized Datasets

In [ ]:
fp = open(train_path+"Chat_Training_Model1.txt", "r", encoding="utf-8")    
other_comments = list()

line = fp.readline()
while(len(line) != 0):
    if "__label__0" in line:
        other_comments.append(line[len("__label__0"):])
    line = fp.readline()
fp.close()


fp = open(train_path+"Chat_Training_Model2.txt", "r", encoding="utf-8")
inappropriate_comments = list()
sexist_comments = list()

line = fp.readline()
while(len(line) != 0):
    if "__label__0" in line:
        inappropriate_comments.append(line[len("__label__0"):])
    elif "__label__1" in line:
        sexist_comments.append(line[len("__label__1"):])
    line = fp.readline()
fp.close()

##### Sexist Comments

In [ ]:
print("Number of sexist comments: ",len(sexist_comments))
print("Corresponds to a",len(sexist_comments)/(len(sexist_comments)+len(inappropriate_comments)+len(other_comments)),"% of the comments in the file.\n")
for comment in sexist_comments:
  print(comment)

##### Inappropiate Comments


In [ ]:
print("Number of inappropiate comments: ",len(inappropriate_comments))
print("Corresponds to a",len(inappropriate_comments)/(len(sexist_comments)+len(inappropriate_comments)+len(other_comments)),"% of the comments in the file.\n")
for comment in inappropriate_comments:
  print(comment)

### FastText model

##### Training

In [ ]:
hyper_params = {"lr": 0.01,
    "epoch": 200,
    "wordNgrams": 2}  

model1 = fasttext.train_supervised(input=train_path+"Chat_Training_Model1.txt", **hyper_params)
model1.save_model(results_path+"model1.bin")

model2 = fasttext.train_supervised(input=train_path+"Chat_Training_Model2.txt", **hyper_params)
model2.save_model(results_path+"model2.bin")

##### Testing

In [ ]:
model1 = fasttext.load_model(results_path+"model1.bin")
model1.test(test_path+"Chat_Testing_Model1.txt")
model2 = fasttext.load_model(results_path+"model2.bin")
model2.test(test_path+"Chat_Testing_Model2.txt")

#### Manual Testing

In [ ]:
model = fasttext.load_model(results_path+"model1.bin")
model.predict(cleanComment("te gusta el sado o mas de chill"))